# Estudo Comparativo entre Sistemas de Recomendação Tradicionais e Baseados em Deep Learning

Este notebook implementa e compara modelos de recomendação tradicionais (FunkSVD, TF-IDF) e baseados em deep learning (AutoRec, DistilRoBERTa) utilizando o The Movies Dataset. Os dataframes utilizados são movies_metadata.csv, ratings.csv e keywords.csv

Objetivos:
- Avaliar desempenho preditivo (Precision@10, Recall@10, NDCG@10)
- Avaliar eficiência computacional (tempo de treino, tempo de inferência, pico de memória RAM)


## Instalação inicial (após as instalações, reiniciar a sessão)

In [ ]:
# Instala versões específicas das bibliotecas numpy e scikit-surprise
!pip install -q numpy==1.26.4 scikit-surprise==1.1.4

In [ ]:
# Baixa o modelo de linguagem en_core_web_lg
!python -m spacy download en_core_web_lg

## Conexão com Google Drive

In [ ]:
# Conectando o Google Colab ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

## Funções/classe executadas em cada modelo

In [1]:
# ===============================
# Pico de memória RAM
# ===============================

import psutil
import threading
import time

# Classe para monitorar o pico de memória RAM
class PeakMemory:
    def __init__(self, interval=0.1, verbose=True):
        self.interval = interval
        self.verbose = verbose
        self.peak_memory = 0
        self.monitoring = False
        self._thread = None
        self.proc = psutil.Process()

    # Método privado que será executado em uma thread separada
    def _monitor(self):
        while self.monitoring:
            try:
                mem_bytes = self.proc.memory_info().rss
                mem_gb = mem_bytes / 1024**3
                self.peak_memory = max(self.peak_memory, mem_gb)
            except (psutil.NoSuchProcess, psutil.AccessDenied):
                pass
            time.sleep(self.interval)

    # Função que inicia o monitoramento de memória RAM
    def start(self):
        if not self.monitoring:
            self.peak_memory = 0
            self.monitoring = True
            self._thread = threading.Thread(target=self._monitor, daemon=True)
            self._thread.start()
            if self.verbose:
                print("Monitoramento da RAM iniciado.")

    # Função que encerra o monitoramento e retorna o pico de memória
    def stop(self):
        if self.monitoring:
            self.monitoring = False
            self._thread.join()
            if self.verbose:
                print("Monitoramento da RAM finalizado:")
                print(f"Pico de memória: {self.peak_memory:.4f} GB")
            return self.peak_memory
        else:
            if self.verbose:
                print("Monitoramento não está ativo.")
            return self.peak_memory

In [2]:
# ===============================
# Funções auxiliares
# ===============================

import pandas as pd
import ast

# Função que extrai os valores da chave 'name' de uma string que representa uma lista de dicionários
def extract_names(x):
    if pd.isna(x):
        return ''
    try:
        dict_list = ast.literal_eval(x)
        return ', '.join(item['name'].strip() for item in dict_list if isinstance(item, dict) and 'name' in item)
    except:
        return ''

# Filtro iterativo que define o número mínimo de interações por usuário e por filme.
def iterative_filter(df, user_min=1, item_min=1):
    prev_shape = None
    while prev_shape != df.shape:
        prev_shape = df.shape
        user_counts = df['userId'].value_counts()
        item_counts = df['movieId'].value_counts()
        valid_users = user_counts[user_counts >= user_min].index
        valid_items = item_counts[item_counts >= item_min].index
        df = df[df['userId'].isin(valid_users) & df['movieId'].isin(valid_items)]
    return df

# Função que pré-processa os dados do dataset
def load_and_process_data(user_min=1, item_min=1):
    # Carregando os dados (Modificar os caminhos caso necessário)
    movies = pd.read_csv("/content/drive/MyDrive/the-movies-dataset/movies_metadata.csv", low_memory=False)
    ratings = pd.read_csv("/content/drive/MyDrive/the-movies-dataset/ratings.csv")
    keywords = pd.read_csv("/content/drive/MyDrive/the-movies-dataset/keywords.csv")

    # Filtrando filmes com id, título e sinopse válidos
    movies = movies.dropna(subset=['id', 'title', 'overview'])
    movies = movies[movies['id'].astype(str).str.isdigit()]
    movies['id'] = movies['id'].astype(int)
    keywords['id'] = keywords['id'].astype(int)

    # Merge com palavras-chave
    movies = movies.merge(keywords[['id', 'keywords']], on='id', how='left')

    # Extração e limpeza de campos de texto
    movies['genres'] = movies['genres'].apply(extract_names)
    movies['keywords'] = movies['keywords'].apply(extract_names)

    # Renomear e filtrar apenas filmes presentes em ratings
    movies.rename(columns={'id': 'movieId'}, inplace=True)
    valid_ids = set(ratings['movieId'].unique())
    movies = movies[movies['movieId'].isin(valid_ids)].drop_duplicates('movieId')

    # Criar coluna com o texto formado (title + genres + keywords + overview)
    movies['text'] = (
        "Title: " + movies['title'] + ". " +
        "Genres: " + movies['genres'] + ". " +
        "Keywords: " + movies['keywords'] + ". " +
        "Overview: " + movies['overview']
    )

    # Processar ratings
    ratings = ratings.drop_duplicates(['userId', 'movieId'])
    ratings = ratings[ratings['movieId'].isin(movies['movieId'])]
    ratings = iterative_filter(ratings, user_min, item_min)

    # Resetar índices e return
    return movies.reset_index(drop=True), ratings.reset_index(drop=True)

## FunkSVD

In [ ]:
# =================================
# Funções auxiliares para o FunkSVD
# =================================

import numpy as np
from collections import defaultdict

# Função que divide as interações em treino e teste por usuário
def train_test_split_per_user(df, test_size=0.2, seed=42):
    train_list = []
    test_list = []

    grouped = df.groupby('userId')
    for user_id, group in grouped:
        if len(group) >= 5:
            test_group = group.sample(frac=test_size, random_state=seed)
            train_group = group.drop(test_group.index)
            test_list.append(test_group)
            train_list.append(train_group)
        else:
            train_list.append(group)

    df_train = pd.concat(train_list, ignore_index=True)
    df_test = pd.concat(test_list, ignore_index=True)
    return df_train, df_test

# Função que agrupa as predições por usuário e retorna os top-N itens recomendados
def get_top_n(predictions, n=10, movies_watched=None):
    top_n = defaultdict(list)

    # Itera sobre todas as predições fornecidas
    for uid, iid, true_r, est, _ in predictions:
        if movies_watched and iid in movies_watched.get(uid, set()):
            continue  # pula itens já avaliados
        top_n[uid].append((iid, est))

    # Para cada usuário, ordena os itens pela nota estimada (est) em ordem decrescente
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# Função que calcula as métricas Precision@k, Recall@k e NDCG@k
def evaluate_metrics(top_n, testset, threshold=4.0, k=10):
    # Agrupar itens relevantes por usuário
    true_ratings = defaultdict(set)
    for uid, iid, true_r in testset:
        if true_r >= threshold:
            true_ratings[uid].add(iid)

    precisions, recalls, ndcgs = [], [], []

    log2 = np.log2

    # Loop por usuário para calcular as métricas com base nas recomendações em top_n
    for uid, user_top in top_n.items():
        relevant = true_ratings.get(uid) # Itens realmente relevantes para o usuário
        if not user_top or not relevant:
            continue

        recommended = [iid for iid, _ in user_top[:k]]

        hits = [iid in relevant for iid in recommended]  # lista booleana
        num_hits = sum(hits) # número de itens relevantes corretamente recomendados

        # Precision@k
        precision = num_hits / k

        # Recall@k
        recall = num_hits / len(relevant)

        # NDCG@k
        dcg = sum(hit / log2(idx + 2) for idx, hit in enumerate(hits))
        idcg = sum(1.0 / log2(i + 2) for i in range(min(len(relevant), k)))
        ndcg = dcg / idcg if idcg > 0 else 0.0

        precisions.append(precision)
        recalls.append(recall)
        ndcgs.append(ndcg)

    # Calcula a média das métricas entre todos os usuários avaliados
    mean_precision = np.mean(precisions) if precisions else 0.0
    mean_recall = np.mean(recalls) if recalls else 0.0
    mean_ndcg = np.mean(ndcgs) if ndcgs else 0.0

    return mean_precision, mean_recall, mean_ndcg

In [ ]:
# Carregar dados de filmes e avaliações
movies, ratings = load_and_process_data(100, 100)

# Merge entre os dataframes
df = ratings.merge(movies, on='movieId', how='inner')

# Seleção colunas relevantes
df = df[['userId', 'movieId', 'rating', 'title']]

# Resetar índices
df.reset_index(drop=True, inplace=True)

# Informações sobre o dataframe resultante
print(f"Dataframe: Usuários = {df['userId'].nunique()}, Filmes = {df['movieId'].nunique()}, Avaliações = {len(df)}\n")
print(f"Valores ausentes por coluna:\n{df.isnull().sum()}\n")
df.head()

Dataframe: Usuários = 28817, Filmes = 2827, Avaliações = 5949128

Valores ausentes por coluna:
userId     0
movieId    0
rating     0
title      0
dtype: int64



,userId,movieId,rating,title
0,12,16,4.0,Dancer in the Dark
1,12,17,4.0,The Dark
2,12,73,2.0,American History X
3,12,82,4.0,Miami Vice
4,12,97,4.0,Tron


In [ ]:
import gc

# Limpeza das variáveis globais que não mais necessárias
del movies, ratings
gc.collect()

In [ ]:
from surprise import Dataset, Reader, SVD

# Listas para guardar métricas
precisions, recalls, ndcgs = [], [], []
train_times, inference_times, peak_memories = [], [], []

# Avaliação das métricas em 5 repetições
for rep in range(5):

    # Monitor de memória
    monitor = PeakMemory(0.1, False)
    monitor.start()

    # Divisão das interações por usuário (com semente diferente a cada repetição)
    train, test = train_test_split_per_user(df, test_size=0.2, seed=42 + rep)

    # Preparar os dados para o Surprise
    reader = Reader(rating_scale=(1.0, 5.0))
    train = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)
    trainset = train.build_full_trainset()
    testset = list(test[['userId', 'movieId', 'rating']].itertuples(index=False, name=None))

    # Criar e treinar o modelo
    model = SVD()
    start_time = time.time()
    model.fit(trainset)
    train_time = time.time() - start_time

    # Cria um dicionário que mapeia cada usuário para o conjunto de filmes que ele já assistiu no conjunto de treino.
    movies_watched = {
        trainset.to_raw_uid(u): set(trainset.to_raw_iid(i) for (i, _) in trainset.ur[u])
        for u in trainset.all_users()
    }

    start_inference = time.time()

    # Fazer predições
    predictions = model.test(testset)

    # Processa as predições para gerar top-N recomendados
    top_n = get_top_n(predictions, n=10, movies_watched)

    inference_time = time.time() - start_inference

    # Métricas
    precision, recall, ndcg = evaluate_metrics(top_n, testset)

    # Armazenar métricas
    precisions.append(precision)
    recalls.append(recall)
    ndcgs.append(ndcg)
    train_times.append(train_time)
    inference_times.append(inference_time / len(top_n))
    peak_memory = monitor.stop()
    peak_memories.append(peak_memory) # Parar monitoramento

    # Limpar variáveis e forçar coleta de lixo
    del (
      model, train, test, trainset, testset, predictions, top_n, movies_watched, monitor, reader, start_time,
      start_inference, train_time, inference_time, precision, recall, ndcg
    )
    gc.collect()


# Resultados
print("\n===== Resultados =====")
print(f"Precision@10:         {np.mean(precisions):.4f} ± {np.std(precisions):.4f}")
print(f"Recall@10:            {np.mean(recalls):.4f} ± {np.std(recalls):.4f}")
print(f"NDCG@10:              {np.mean(ndcgs):.4f} ± {np.std(ndcgs):.4f}\n")
print(f"Tempo de treinamento: {np.mean(train_times)/60:.2f} ± {np.std(train_times)/60:.2f} min")
print(f"Tempo de inferência:  {np.mean(inference_times)*1000:.6f} ± {np.std(inference_times)*1000:.6f} ms")
print(f"Pico de memória RAM:  {np.mean(peak_memories):.4f} ± {np.std(peak_memories):.4f} GB")



===== Resultados =====
Precision@10:         0.7365 ± 0.0014
Recall@10:            0.4871 ± 0.0008
NDCG@10:              0.7952 ± 0.0012

Tempo de treinamento: 0.74 ± 0.00 min
Tempo de inferência:  0.349520 ± 0.008666 ms
Pico de memória RAM:  2.9276 ± 0.0008 GB


## TF-IDF

### Reiniciar a sessão e executar novamente as funções/classe iniciais

In [ ]:
import re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import numpy as np

# Download de recursos do NLTK
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')

# Função que calcula as métricas de desempenho preditivo
def metrics(recommended, relevant, k=10):
    # Seleciona os k primeiros itens recomendados
    recommended_k = recommended[:k]

    # Converte as listas de recomendados e relevantes para conjuntos (sets)
    recommended_set = set(recommended_k)
    relevant_set = set(relevant)

    # Encontra os itens que foram recomendados e que também são relevantes
    hits = recommended_set & relevant_set
    hits_count = len(hits)

    # Calcula a precisão
    precision = hits_count / k if k > 0 else 0.0

    # Calcula o recall
    recall = hits_count / len(relevant) if relevant else 0.0

    # Calcula o NDCG
    relevance = [1 if item in relevant_set else 0 for item in recommended_k]

    def dcg(scores):
        return sum(rel / np.log2(idx + 2) for idx, rel in enumerate(scores))

    dcg_val = dcg(relevance)
    ideal_relevance = sorted(relevance, reverse=True)
    idcg_val = dcg(ideal_relevance)
    ndcg = dcg_val / idcg_val if idcg_val != 0 else 0.0

    return precision, recall, ndcg

# Função usada para melhorar a lematização com base na classe gramatical da palavra
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # padrão

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Pré-processamento do texto
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = nltk.word_tokenize(text)
    tokens = [t for t in tokens if t not in stop_words]
    pos_tags = pos_tag(tokens)
    tokens = [lemmatizer.lemmatize(t, get_wordnet_pos(p)) for t, p in pos_tags]
    return ' '.join(tokens)

In [ ]:
# Carregar dados
movies, ratings = load_and_process_data(100, 100)

# Aplica pré-processamento do texto
movies['text'] = movies['text'].astype(str).apply(preprocess_text)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Vetorização TF-IDF
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.8
)
tfidf_matrix = tfidf.fit_transform(movies['text'])

# Similaridade do cosseno
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

dataframe_tfidf = pd.DataFrame(
    cosine_similarities,
    index=movies['movieId'],
    columns=movies['movieId']
)

In [ ]:
# Merge entre os dataframes
df = ratings.merge(movies, on='movieId', how='inner')

# Seleção colunas relevantes
df = df[['userId', 'movieId', 'rating', 'title']]

# Resetar índices
df.reset_index(drop=True, inplace=True)

# Informações sobre o dataframe resultante
print(f"Dataframe: Usuários = {df['userId'].nunique()}, Filmes = {df['movieId'].nunique()}, Avaliações = {len(df)}\n")
print(f"Valores ausentes por coluna:\n{df.isnull().sum()}\n")
df.head()

Dataframe: Usuários = 28817, Filmes = 2827, Avaliações = 5949128

Valores ausentes por coluna:
userId     0
movieId    0
rating     0
title      0
dtype: int64



,userId,movieId,rating,title
0,12,16,4.0,Dancer in the Dark
1,12,17,4.0,The Dark
2,12,73,2.0,American History X
3,12,82,4.0,Miami Vice
4,12,97,4.0,Tron


In [ ]:
# Obtém uma lista de IDs de usuários ordenados pelo número de interações
user_counts = df['userId'].value_counts()
user_list = user_counts.index.tolist()

# Pré-conversão do DataFrame para array para acesso rápido
movie_ids = dataframe_tfidf.index.tolist()
movie_id_to_idx = {movie_id: idx for idx, movie_id in enumerate(movie_ids)}
similarity_matrix = dataframe_tfidf.to_numpy()

In [ ]:
import gc

# Limpeza das variáveis globais que não mais necessárias
del (
    movies, ratings, user_counts, stop_words, lemmatizer, tfidf, tfidf_matrix,
    cosine_similarities, dataframe_tfidf, movie_ids
)
gc.collect()

In [ ]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Listas para resultados por repetição
precisions = []
recalls = []
ndcgs = []
inference_times = []
peak_memories = []

# Avaliação das métricas em 5 repetições
for rep in range(5):

    # Inicia o monitoramento do pico de uso de memória RAM
    monitor = PeakMemory(0.1, False)
    monitor.start()

    # Inicializa listas para armazenar as métricas dessa repetição
    precision_scores = []
    recall_scores = []
    ndcg_scores = []
    inference_scores = []

    # Itera sobre todos os usuários da lista, exibindo barra de progresso
    for user in tqdm(user_list, desc=f"Execução {rep + 1} - Processando usuários"):
        try:
            # Seleciona todas as avaliações do usuário atual no dataframe
            user_ratings = df[df['userId'] == user]

            # Divide as avaliações em treino e teste
            train, test = train_test_split(user_ratings, test_size=0.2, random_state=42 + rep)

            # Considera apenas avaliações altas (rating >= 4) para treino e teste
            train_high = train[train['rating'] >= 4]
            test_high = test[test['rating'] >= 4]

            # Se não houver avaliações altas nem no treino nem no teste, pula o usuário
            if train_high.empty or test_high.empty:
                continue

            # Converte os IDs dos filmes em índices usados na matriz de similaridade para treino
            train_indices = [movie_id_to_idx[mid] for mid in train_high['movieId'] if mid in movie_id_to_idx]

            # Lista de filmes do conjunto de teste
            test_movies = test['movieId'].tolist()

            # Índices dos filmes do teste na matriz de similaridade
            test_indices = [movie_id_to_idx[mid] for mid in test_movies if mid in movie_id_to_idx]

            # Se algum dos índices estiver vazio, pula o usuário
            if not train_indices or not test_indices:
                continue

            inference_start = time.time()

            # Calcula o perfil do usuário como a média das linhas da matriz de similaridade dos filmes do treino
            user_profile = similarity_matrix[train_indices].mean(axis=0)

            # Pega os scores do perfil para os filmes do conjunto de teste
            scores = user_profile[test_indices]

            # Junta os filmes de teste com seus scores
            movie_scores = list(zip([test_movies[i] for i in range(len(test_indices))], scores))

            # Ordena os filmes de teste por score decrescente
            movie_scores.sort(key=lambda x: x[1], reverse=True)

            # Seleciona os top 10 filmes recomendados
            recommended = [mid for mid, _ in movie_scores[:10]]

            # Calcula o tempo gasto para inferência e armazena
            inference_end = time.time()
            inference_time = inference_end - inference_start
            inference_scores.append(inference_time)

            # Lista de filmes relevantes no conjunto de teste
            relevant = test_high['movieId'].tolist()

            # Calcula as métricas de avaliação para as recomendações geradas
            prec, rec, ndcg = metrics(recommended, relevant, 10)

            # Armazena as métricas para essa repetição e usuário
            precision_scores.append(prec)
            recall_scores.append(rec)
            ndcg_scores.append(ndcg)

        except Exception as e:
            print(f"Erro com usuário {user}: {e}")
            continue


    # Armazena as medidas
    precisions.append(np.mean(precision_scores))
    recalls.append(np.mean(recall_scores))
    ndcgs.append(np.mean(ndcg_scores))
    inference_times.append(np.mean(inference_scores))
    peak_memory = monitor.stop() # Finaliza monitoramento de RAM
    peak_memories.append(peak_memory)

    print("")

    # Limpeza de variáveis e coleta de lixo
    del (
        precision_scores, recall_scores, ndcg_scores, inference_scores, monitor, peak_memory
    )
    gc.collect()


# Resultados
print("\n===== Resultados =====")
print(f"Precision@10:         {np.mean(precisions):.4f} ± {np.std(precisions):.4f}")
print(f"Recall@10:            {np.mean(recalls):.4f} ± {np.std(recalls):.4f}")
print(f"NDCG@10:              {np.mean(ndcgs):.4f} ± {np.std(ndcgs):.4f}\n")
print(f"Tempo de inferência:  {np.mean(inference_times)*1000:.6f} ± {np.std(inference_times)*1000:.6f} ms")
print(f"Pico de memória RAM:  {np.mean(peak_memories):.4f} ± {np.std(peak_memories):.4f} GB")


Execução 1 - Processando usuários: 100%|██████████| 28817/28817 [02:50<00:00, 168.89it/s]


Execução 2 - Processando usuários: 100%|██████████| 28817/28817 [02:49<00:00, 169.78it/s]


Execução 3 - Processando usuários: 100%|██████████| 28817/28817 [02:49<00:00, 169.58it/s]


Execução 4 - Processando usuários: 100%|██████████| 28817/28817 [02:50<00:00, 169.19it/s]


Execução 5 - Processando usuários: 100%|██████████| 28817/28817 [02:50<00:00, 169.34it/s]




===== Resultados =====
Precision@10:         0.4898 ± 0.0012
Recall@10:            0.3064 ± 0.0008
NDCG@10:              0.7328 ± 0.0006

Tempo de inferência:  1.117377 ± 0.005708 ms
Pico de memória RAM:  1.0999 ± 0.0129 GB


## AutoRec

### Reiniciar a sessão e executar novamente as funções/classe iniciais

In [ ]:
# =================================
# Funções auxiliares para o AutoRec
# =================================

import numpy as np
from collections import defaultdict

# Função que divide as interações em treino e teste por usuário
def train_test_split_per_user(df, test_size=0.2, seed=42):
    train_list = []
    test_list = []

    grouped = df.groupby('userId')
    for user_id, group in grouped:
        if len(group) >= 5:
            test_group = group.sample(frac=test_size, random_state=seed)
            train_group = group.drop(test_group.index)
            test_list.append(test_group)
            train_list.append(train_group)
        else:
            train_list.append(group)

    df_train = pd.concat(train_list, ignore_index=True)
    df_test = pd.concat(test_list, ignore_index=True)
    return df_train, df_test

# Função que agrupa as predições por usuário e retorna os top-N itens recomendados
def get_top_n(predictions, seen_items_by_user, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est in predictions:
        if iid not in seen_items_by_user.get(uid, set()):
            top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

# Função que calcula as métricas Precision@k, Recall@k e NDCG@k
def evaluate_metrics(top_n, testset, threshold=4.0, k=10):
    # Agrupar itens relevantes por usuário
    true_ratings = defaultdict(set)
    for uid, iid, true_r in testset:
        if true_r >= threshold:
            true_ratings[uid].add(iid)

    precisions, recalls, ndcgs = [], [], []

    log2 = np.log2

    # Loop por usuário para calcular as métricas com base nas recomendações em top_n
    for uid, user_top in top_n.items():
        relevant = true_ratings.get(uid) # Itens realmente relevantes para o usuário
        if not user_top or not relevant:
            continue

        recommended = [iid for iid, _ in user_top[:k]]

        hits = [iid in relevant for iid in recommended]  # lista booleana
        num_hits = sum(hits) # número de itens relevantes corretamente recomendados

        # Precision@k
        precision = num_hits / k

        # Recall@k
        recall = num_hits / len(relevant)

        # NDCG@k
        dcg = sum(hit / log2(idx + 2) for idx, hit in enumerate(hits))
        idcg = sum(1.0 / log2(i + 2) for i in range(min(len(relevant), k)))
        ndcg = dcg / idcg if idcg > 0 else 0.0

        precisions.append(precision)
        recalls.append(recall)
        ndcgs.append(ndcg)

    # Calcula a média das métricas entre todos os usuários avaliados
    mean_precision = np.mean(precisions) if precisions else 0.0
    mean_recall = np.mean(recalls) if recalls else 0.0
    mean_ndcg = np.mean(ndcgs) if ndcgs else 0.0

    return mean_precision, mean_recall, mean_ndcg

In [ ]:
# Carregar dados de filmes e avaliações
movies, ratings = load_and_process_data(100, 100)

# Merge entre os dataframes
df = ratings.merge(movies, on='movieId', how='inner')

# Seleção colunas relevantes
df = df[['userId', 'movieId', 'rating', 'title']]

# Resetar índices
df.reset_index(drop=True, inplace=True)

# Informações sobre o dataframe resultante
print(f"Dataframe: Usuários = {df['userId'].nunique()}, Filmes = {df['movieId'].nunique()}, Avaliações = {len(df)}\n")
print(f"Valores ausentes por coluna:\n{df.isnull().sum()}\n")
df.head()

Dataframe: Usuários = 28817, Filmes = 2827, Avaliações = 5949128

Valores ausentes por coluna:
userId     0
movieId    0
rating     0
title      0
dtype: int64



,userId,movieId,rating,title
0,12,16,4.0,Dancer in the Dark
1,12,17,4.0,The Dark
2,12,73,2.0,American History X
3,12,82,4.0,Miami Vice
4,12,97,4.0,Tron


In [ ]:
# Definindo os hiperparâmetros
encoding_dim = 500
l2_lambda = 1e-5
dropout_rate = 0.1
learning_rate = 0.001

In [ ]:
import gc

# Limpeza das variáveis globais que não mais necessárias
del movies, ratings
gc.collect()

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Armazenar resultados
precision_scores = []
recall_scores = []
ndcg_scores = []
train_times = []
inference_times = []
peak_memories = []

# Avaliação das métricas em 5 repetições
for rep in range(5):

    # Inicia o monitoramento do pico de uso de memória RAM
    monitor = PeakMemory(0.1, False)
    monitor.start()

    # Define sementes para reprodutibilidade
    np.random.seed(42 + rep)
    tf.random.set_seed(42 + rep)

    # Divisão treino/teste
    df_train, df_test = train_test_split_per_user(df, test_size=0.2, seed=42 + rep)

    # Mapeamento dos IDs de usuários e itens para índices
    user_ids = pd.concat([df_train['userId'], df_test['userId']]).unique()
    item_ids = pd.concat([df_train['movieId'], df_test['movieId']]).unique()
    user2idx = {uid: i for i, uid in enumerate(user_ids)}
    item2idx = {iid: i for i, iid in enumerate(item_ids)}
    num_users, num_items = len(user_ids), len(item_ids)

    # Inicialização das matrizes de avaliação (com todas as avaliações iniciadas como 0)
    R_train = np.zeros((num_users, num_items), dtype=np.float32)
    R_test = np.zeros((num_users, num_items), dtype=np.float32)
    mask_train = np.zeros((num_users, num_items), dtype=np.float32)
    mask_test = np.zeros((num_users, num_items), dtype=np.float32)

    # Preenchendo as matrizes de treino e teste com as avaliações reais
    for u, i, r in zip(df_train['userId'], df_train['movieId'], df_train['rating']):
        R_train[user2idx[u], item2idx[i]] = r
        mask_train[user2idx[u], item2idx[i]] = 1.0

    for u, i, r in zip(df_test['userId'], df_test['movieId'], df_test['rating']):
        R_test[user2idx[u], item2idx[i]] = r
        mask_test[user2idx[u], item2idx[i]] = 1.0

    # Definição do modelo User-based com TensorFlow
    input_ratings = tf.keras.Input(shape=(num_items,))
    input_mask = tf.keras.Input(shape=(num_items,))

    # Camadas do modelo: uma camada densa com ReLU e regularização L2
    x = tf.keras.layers.Dense(encoding_dim, activation='relu',
                              kernel_regularizer=tf.keras.regularizers.l2(l2_lambda))(input_ratings)
    x = tf.keras.layers.Dropout(dropout_rate)(x)  # Dropout para evitar overfitting
    decoded = tf.keras.layers.Dense(num_items, activation='linear',
                                    kernel_regularizer=tf.keras.regularizers.l2(l2_lambda))(x)

    # Multiplicação do resultado com a máscara para ignorar itens não avaliados
    output = tf.keras.layers.Multiply()([decoded, input_mask])

    # Criação e compilação do modelo
    model = tf.keras.Model(inputs=[input_ratings, input_mask], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')

    # Treinamento do modelo
    y_train = R_train * mask_train  # Apenas as avaliações que foram feitas são usadas
    start_time = time.time()
    model.fit([R_train, mask_train], y_train, epochs=30, batch_size=32, verbose=0)
    training_time = time.time() - start_time

    # Previsões para o conjunto de teste
    start_inference = time.time()
    R_pred = model.predict([R_test, mask_test], verbose=0)
    user_indices = np.where(mask_test)[0]

    # Avaliação do modelo
    seen_items_by_user = defaultdict(set)  # Dicionário para armazenar os itens vistos por cada usuário
    for u, i in zip(df_train['userId'], df_train['movieId']):
        seen_items_by_user[user2idx[u]].add(item2idx[i])

    # Preparação para calcular as métricas de avaliação
    predictions = []
    testset = []
    user_indices, item_indices = np.where(mask_test)
    for u, i in zip(user_indices, item_indices):
        predictions.append((u, i, R_test[u, i], R_pred[u, i]))  # Previsão feita pelo modelo
        testset.append((u, i, R_test[u, i]))  # Rótulo verdadeiro

    # Obter as N recomendações principais para cada usuário
    top_n = get_top_n(predictions, seen_items_by_user, n=10)

    # Cálculo do tempo de inferência
    end_inference = time.time()
    inference_time = (end_inference - start_inference) / len(top_n)

    # Avaliação das métricas de desempenho (precisão, recall, NDCG)
    precision, recall, ndcg = evaluate_metrics(top_n, testset)

    # Armazenamento das métricas e resultados
    precision_scores.append(precision)
    recall_scores.append(recall)
    ndcg_scores.append(ndcg)
    train_times.append(training_time)
    inference_times.append(inference_time)
    peak_memory = monitor.stop()  # Parar o monitoramento de memória RAM
    peak_memories.append(peak_memory)

    # Limpeza de memória
    del (
      df_train, df_test, R_train, R_test, mask_train, mask_test,
      input_ratings, input_mask, x, decoded, output, model,
      predictions, testset, top_n, seen_items_by_user, R_pred, y_train,
      start_time, start_inference, end_inference, training_time, inference_time,
      user2idx, item2idx, user_ids, item_ids, user_indices, item_indices,
      num_users, num_items
    )
    gc.collect()
    tf.keras.backend.clear_session()  # Limpar a sessão do Keras para liberar memória


# Resultados
print("\n===== Resultados =====")
print(f"Precision@10:         {np.mean(precision_scores):.4f} ± {np.std(precision_scores):.4f}")
print(f"Recall@10:            {np.mean(recall_scores):.4f} ± {np.std(recall_scores):.4f}")
print(f"NDCG@10:              {np.mean(ndcg_scores):.4f} ± {np.std(ndcg_scores):.4f}\n")
print(f"Tempo de treinamento: {np.mean(train_times)/60:.2f} ± {np.std(train_times)/60:.2f} min")
print(f"Tempo de inferência:  {np.mean(inference_times)*1000:.6f} ± {np.std(inference_times)*1000:.6f} ms")
print(f"Pico de memória RAM:  {np.mean(peak_memories):.4f} ± {np.std(peak_memories):.4f} GB")



===== Resultados =====
Precision@10:         0.7235 ± 0.0026
Recall@10:            0.4785 ± 0.0012
NDCG@10:              0.7841 ± 0.0023

Tempo de treinamento: 0.99 ± 0.01 min
Tempo de inferência:  0.361330 ± 0.050205 ms
Pico de memória RAM:  5.8986 ± 0.1001 GB


## DistilRoBERTa

### Reiniciar a sessão e executar novamente as funções/classe iniciais

In [3]:
# =======================================
# Funções auxiliares para o DistilRoBERTa
# =======================================

import random
from tqdm import tqdm
from sentence_transformers import InputExample
from itertools import combinations

# Função que calcula a precision@k
def precision_at_k(recommended, relevant, k=10):
    recommended = list(recommended)
    relevant = list(relevant)

    if len(recommended) == 0:
        return 0.0

    recommended_k = recommended[:k]
    hits = len(set(recommended_k) & set(relevant))
    return hits / min(k, len(recommended))

# Função que calcula a recall@k
def recall_at_k(recommended, relevant, k=10):
    recommended = list(recommended)
    relevant = list(relevant)

    if len(relevant) == 0:
        return 0.0

    recommended_k = recommended[:k]
    hits = len(set(recommended_k) & set(relevant))
    return hits / len(relevant)

# Função que calcula NDCG@k
def ndcg_at_k(recommended, relevant, k=10):
    recommended = list(recommended)
    relevant = set(relevant)

    recommended_k = recommended[:k]
    relevance = [1 if item in relevant else 0 for item in recommended_k]

    def dcg(scores):
        return sum(rel / np.log2(idx + 2) for idx, rel in enumerate(scores))

    dcg_val = dcg(relevance)
    ideal_relevance = sorted(relevance, reverse=True)
    idcg_val = dcg(ideal_relevance)

    return dcg_val / idcg_val if idcg_val != 0 else 0.0


# Função que gera os triplets
def generate_triplets(df_movies, min_positive=2, max_triplets_per_user=50, max_users=200, seed=42):
    random.seed(seed)  # Define a semente para gerar números aleatórios

    # Filtra os dados para selecionar um número limitado de usuários (max_users)
    users = df_movies['userId'].unique()
    random.shuffle(users)
    users = users[:max_users]

    # Filtra o dataframe para manter apenas os dados dos usuários selecionados.
    df_movies = df_movies[df_movies['userId'].isin(users)]
    triplets = []

    # Itera sobre cada usuário para gerar os triplets.
    for user in tqdm(users, desc="Generating triplets"):
        group = df_movies[df_movies['userId'] == user]  # Filtra o dataframe para pegar apenas as avaliações do usuário.

        # Separa as avaliações positivas (nota >= 4) e negativas (nota < 3).
        positives = group[group['rating'] >= 4]['text'].tolist()
        negatives = group[group['rating'] < 3]['text'].tolist()

        # Se o usuário não tiver avaliações positivas suficientes ou não tiver avaliações negativas, ignora o usuário.
        if len(positives) < min_positive or len(negatives) < 1:
            continue

        # Cria todas as combinações possíveis de pares de filmes positivos.
        combinations_list = list(combinations(positives, 2))
        random.shuffle(combinations_list)
        combinations_list = combinations_list[:max_triplets_per_user]  # Limita o número de combinações a max_triplets_per_user.

        # Para cada par de filmes positivos, escolhe aleatoriamente um filme negativo e cria um triplet.
        for anchor, positive in combinations_list:
            negative = random.choice(negatives)
            triplets.append(InputExample(texts=[anchor, positive, negative]))  # Adiciona o triplet à lista de triplets.

    return triplets

In [4]:
import spacy

# Carregar o modelo de linguagem do spaCy
nlp = spacy.load("en_core_web_lg")

# Carregar dados de filmes e avaliações
movies, ratings = load_and_process_data(100, 100)

# Merge entre os dataframes
df = ratings.merge(movies, on='movieId', how='inner')

# Seleção colunas relevantes
df = df[['userId', 'movieId', 'rating', 'title', 'text']]

# Resetar índices
df.reset_index(drop=True, inplace=True)

# Informações sobre o dataframe resultante
print(f"Dataframe: Usuários = {df['userId'].nunique()}, Filmes = {df['movieId'].nunique()}, Avaliações = {len(df)}\n")
print(f"Valores ausentes por coluna:\n{df.isnull().sum()}\n")
df.head()

Dataframe: Usuários = 28817, Filmes = 2827, Avaliações = 5949128

Valores ausentes por coluna:
userId     0
movieId    0
rating     0
title      0
text       0
dtype: int64



,userId,movieId,rating,title,text
0,12,16,4.0,Dancer in the Dark,"Title: Dancer in the Dark. Genres: Drama, Crim..."
1,12,17,4.0,The Dark,"Title: The Dark. Genres: Horror, Thriller, Mys..."
2,12,73,2.0,American History X,Title: American History X. Genres: Drama. Keyw...
3,12,82,4.0,Miami Vice,"Title: Miami Vice. Genres: Action, Adventure, ..."
4,12,97,4.0,Tron,"Title: Tron. Genres: Science Fiction, Action, ..."


In [ ]:
# Limpeza das variáveis globais que não mais necessárias
import gc
del ratings

gc.collect()

In [6]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, losses
from tqdm import tqdm

# Desativa logging externo
os.environ["WANDB_DISABLED"] = "true"

# Inicialização
precisions = []
recalls = []
ndcgs = []
train_times = []
inference_times = []
peak_memories = []

# Avaliação das métricas em 5 repetições
for rep in range(5):
    print(f"\n=== Execução {rep+1} ===")

    # Inicia o monitoramento do pico de uso de memória RAM
    monitor = PeakMemory(0.1, False)
    monitor.start()

    # Geração dos dados de treino (gerando triplets)
    triplets = generate_triplets(df, seed=42 + rep)

    print(f"Total de trios gerados: {len(triplets)}\n")

    # Inicializa modelo
    model_triplet = SentenceTransformer('paraphrase-distilroberta-base-v1')
    train_dataloader = DataLoader(triplets, shuffle=True, batch_size=32)
    train_loss = losses.TripletLoss(model=model_triplet)

    # Treinamento com triplet loss
    print("Iniciando o treinamento...")
    start_train = time.time()
    model_triplet.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=10,
        warmup_steps=100,
        show_progress_bar=True,
        optimizer_params={'lr': 5e-5}
    )
    end_train = time.time()
    training_time = end_train - start_train
    train_times.append(training_time)

    # Geração de embeddings
    embeddings_tensor = model_triplet.encode(movies['text'].tolist(), convert_to_tensor=True)
    embedding_df = pd.DataFrame(embeddings_tensor.cpu().numpy())

    # Cálculo da similaridade do cosseno
    cosine_similarities_bert = cosine_similarity(embedding_df, embedding_df)
    similarities_df_bert = pd.DataFrame(
        cosine_similarities_bert,
        index=movies['movieId'],
        columns=movies['movieId']
    )

    # Armazena os resultados de cada usuário
    precision_scores = []
    recall_scores = []
    ndcg_scores = []
    inference_scores = []

    user_counts = df['userId'].value_counts()
    user_list = user_counts.index.tolist()

    # Itera sobre todos os usuários da lista, exibindo barra de progresso
    for user in tqdm(user_list, desc="Processando usuários", unit="usuário"):
        try:
            user_ratings = df[df['userId'] == user]
            train, test = train_test_split(user_ratings, test_size=0.2, random_state=42 + rep)

            train_movies = train[train['rating'] > 4]['movieId'].values
            relevant = test[test['rating'] >= 4]['movieId'].values
            test_movies = test['movieId'].values

            # Ignora usuários sem filmes de treino ou sem relevantes
            if len(train_movies) == 0 or len(relevant) == 0:
                continue

            # Garante que todos os filmes existem no DataFrame de similaridade
            valid_test_movies = [m for m in test_movies if m in similarities_df_bert.index]
            valid_train_movies = [m for m in train_movies if m in similarities_df_bert.columns]

            if len(valid_test_movies) == 0 or len(valid_train_movies) == 0:
                continue

            inference_start = time.time()

            # Calcula a similaridade apenas com filmes válidos e realiza a recomendação
            similarities = similarities_df_bert.loc[valid_test_movies, valid_train_movies].sum(axis=1)
            test_valid = test[test['movieId'].isin(valid_test_movies)].copy()
            test_valid = test_valid.assign(pontuacao=similarities.values)
            recommended = test_valid.sort_values(by='pontuacao', ascending=False)['movieId'].head(10).tolist()

            inference_end = time.time()
            inference_time = inference_end - inference_start
            inference_scores.append(inference_time)

            # Avaliação das métricas
            prec = precision_at_k(recommended, relevant, 10)
            rec = recall_at_k(recommended, relevant, 10)
            ndcg = ndcg_at_k(recommended, relevant, 10)
            precision_scores.append(prec)
            recall_scores.append(rec)
            ndcg_scores.append(ndcg)

        except Exception as e:
            print(f"Erro com usuário {user}: {e}")
            continue

    # Salva métricas
    precisions.append(np.mean(precision_scores))
    recalls.append(np.mean(recall_scores))
    ndcgs.append(np.mean(ndcg_scores))
    inference_times.append(np.mean(inference_scores))
    peak_memory = monitor.stop() # Finaliza o monitoramento da RAM
    peak_memories.append(peak_memory)

    # Limpa memória
    del (
      model_triplet, triplets, train_dataloader, train_loss, embeddings_tensor, embedding_df,
      cosine_similarities_bert, similarities_df_bert, precision_scores, recall_scores,
      ndcg_scores, inference_scores, user_ratings, train, test, train_movies, relevant, test_movies,
      valid_test_movies, valid_train_movies, similarities, test_valid, recommended, monitor,
      start_train, end_train, training_time, inference_start, inference_end, inference_time,
      user_counts, user_list
    )

    gc.collect()
    if torch.cuda.is_available():
      torch.cuda.empty_cache()

# Resultados
print("\n===== Resultados =====")
print(f"Precision@10:         {np.mean(precisions):.4f} ± {np.std(precisions):.4f}")
print(f"Recall@10:            {np.mean(recalls):.4f} ± {np.std(recalls):.4f}")
print(f"NDCG@10:              {np.mean(ndcgs):.4f} ± {np.std(ndcgs):.4f}\n")
print(f"Tempo de treinamento: {np.mean(train_times)/60:.2f} ± {np.std(train_times)/60:.2f} min")
print(f"Tempo de inferência:  {np.mean(inference_times)*1000:.6f} ± {np.std(inference_times)*1000:.6f} ms")
print(f"Pico de memória RAM:  {np.mean(peak_memories):.4f} ± {np.std(peak_memories):.4f} GB")


=== Execução 1 ===


Generating triplets: 100%|██████████| 200/200 [00:00<00:00, 281.21it/s]


Total de trios gerados: 9751



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Iniciando o treinamento...


...

Iniciando o treinamento...


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,2.977900
1000,1.752600
1500,1.300900
2000,0.969700
2500,0.776300
3000,0.597200


Processando usuários: 100%|██████████| 28817/28817 [04:15<00:00, 112.91usuário/s]



===== Resultados =====
Precision@10:         0.6148 ± 0.0052
Recall@10:            0.3948 ± 0.0039
NDCG@10:              0.8336 ± 0.0045

Tempo de treinamento: 22.17 ± 0.21 min
Tempo de inferência:  4.096515 ± 0.008247 ms
Pico de memória RAM:  3.3180 ± 0.0625 GB


## Conclusões

- Modelos baseados em deep learning apresentaram melhor desempenho preditivo, porém com maior custo computacional.
- O TF-IDF teve a menor latência de inferência.
- O AutoRec teve bom equilíbrio entre precisão e desempenho.

Futuros trabalhos podem explorar modelos híbridos ou bases de dados maiores.


## Referências

- The Movies Dataset – https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset  
- AutoRec: Sedhain et al., 2015  
- Transformers – HuggingFace Documentation
